### **Analysis IMDB-Movie-Reviews using BERT and ktrain with model deployment**

---



---



---



### In this notebook, we will use pre-trained deep learning model which is BERT and see how we will use **BERT** to classification problems using KTRAIN library. The text is a list of sentences from film reviews. And we will calssify each sentence as either speaking "positively" about its subject of "negatively".So, In this notebook I'm goona facous on only, how to analysis Moview reviews using BERT also, I will not plot any figure such as word cloud, analysis of most common words presentent in positive sentiment as well as in the negative sentiment so on.

**IMDB Movie Reviews Large Dataset - 50k Reviews**

This dataset is taken from https://ai.stanford.edu/~amaas/data/sentiment/ and then preprocess to put all positive and negative reviews in the same file for training and testing. It help you to put more effort on algorithm instead of data collection.

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import seaborn as sns

### **Check the GPU availability**

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

1 Physical GPUs, 1 Logical GPUs
Version:  2.4.1
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 6.8MB 51.5MB/s 
     |████████████████████████████████| 983kB 47.1MB/s 
     |████████████████████████████████| 266kB 55.7MB/s 
     |████████████████████████████████| 1.9MB 43.3MB/s 
     |████████████████████████████████| 1.2MB 47.9MB/s 
     |████████████████████████████████| 471kB 51.2MB/s 
     |████████████████████████████████| 3.3MB 47.5MB/s 
     |████████████████████████████████| 901kB 50.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=15af68580b1017b1d0550fe5d77954d55acb2b7a7336122fb936d6f4bcadba23
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=96ccf2f393d8444570310e14ea63bcbd162d5f2a913b270224964073c0bb92b6
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### **Load the dataset**

In [5]:
# Read the dataset
train='/content/drive/MyDrive/Google_colab_project/IMDB_train.xlsx'
test='/content/drive/MyDrive/Google_colab_project/IMDB_test.xlsx'
train_movie=pd.read_excel(train,dtype='str')
test_movie=pd.read_excel(test,dtype='str')

In [6]:
test_movie.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


**We will begin by importing some required modules for performing text classification in ktrain.**

In [7]:
import ktrain
from ktrain import text

# **ktrain**

**ktrain** is a library to help build, train, debug, and deploy neural networks in the deep learning software framework, Keras. ... Inspired by the fastai library, with only a few lines of code, ktrain allows you to easily: estimate an optimal learning rate for your model given your data using a learning rate finder.

This is official website for ktrain library. If wanted to get more information about the library please go ahead aand read the information. 
https://libraries.io/pypi/ktrain

Next, we will load **and preprocess the text data for training and validation**. ktrain can load texts and associated labels from a variety of source:
For **texts_from_csv** and **texts_from_df**, labels can either be multi or one-hot-encoded with one column per class or can be a single column storing integers or string.

In [8]:
(X_train,y_train),(X_test,y_test),preprocess=text.texts_from_df(train_df=train_movie,text_column='Reviews',
                                      label_columns='Sentiment',
                                      val_df=test_movie,
                                      maxlen=500,
                                      preprocess_mode='bert',
                                      ngram_range=1)

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [9]:
print("Shape of the Training dataset is "+ str(X_train[0].shape))

Shape of the Training dataset is (25000, 500)


In [10]:
X_train[0]

array([[ 101, 2043, 1045, ...,    0,    0,    0],
       [ 101, 8210, 4301, ...,    0,    0,    0],
       [ 101, 2339, 2515, ...,    0,    0,    0],
       ...,
       [ 101, 2026, 2643, ...,    0,    0,    0],
       [ 101, 2043, 1045, ..., 1028, 2054,  102],
       [ 101, 2061, 2339, ..., 2024, 6160,  102]])

In [11]:
y_test[0]

array([0., 1.], dtype=float32)

In [12]:
model_bert=text.text_classifier(name='bert',train_data=(X_train,y_train),
                                preproc=preprocess)

Is Multi-Label? False
maxlen is 500
done.


### **Finding the optimal Learning rate**

In [13]:
learner=ktrain.get_learner(model_bert,train_data=(X_train,y_train),
                           val_data=(X_test,y_test),batch_size=6)

In [ ]:
learner.fit(0.01, 1, cycle_len=5, checkpoint_folder='/tmp/saved_weights')

Epoch 1/5
4167/4167 [==============================] - 5082s 1s/step - loss: 3.1746 - accuracy: 0.4947 - val_loss: 1.4840 - val_accuracy: 0.5000
Epoch 2/5
 758/4167 [====>.........................] - ETA: 54:46 - loss: 2.5685 - accuracy: 0.5031

In [23]:
learner.lr_find()


simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
4167/4167 [==============================] - 15s 925us/step - loss: 0.1652 - accuracy: 0.9667


done.
Please invoke the Learner.lr_plot() method to visually inspect the loss plot to help identify the maximal learning rate associated with falling loss.


### **Compile the Model**

In [15]:
learner.fit_onecycle(2e-5,epochs=1,cycle_momentum=True,
                     max_momentum=0.95)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5136s 1s/step - loss: 0.3181 - accuracy: 0.8620 - val_loss: 0.0996 - val_accuracy: 0.9696


In [16]:
learner.validate(class_names=preprocess.get_classes())

              precision    recall  f1-score   support

         neg       0.97      0.97      0.97     12500
         pos       0.97      0.97      0.97     12500

    accuracy                           0.97     25000
   macro avg       0.97      0.97      0.97     25000
weighted avg       0.97      0.97      0.97     25000



array([[12065,   435],
       [  326, 12174]])

In [17]:
predictor = ktrain.get_predictor(learner.model, preprocess)

### **Prediction for newdata set**

In [18]:
new_data=["films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before.",
         "if you can get past the whole comic book thing , you might find another stumbling block in from hell's directors , albert and allen hughes .",
         "when the first stiff turns up , copper peter godley ( robbie coltrane , the world is not enough ) calls in inspector frederick abberline ( johnny depp , blow ) to crack the case ."]

In [19]:
predictor.predict(new_data)

['neg', 'neg', 'pos']

In [20]:
#save the model
predictor.save('/content/drive/My Drive/bert_IDMB')

### **Model evaluation for test dataset**

In [21]:
learner.validate(class_names=preprocess.get_classes())

              precision    recall  f1-score   support

         neg       0.97      0.97      0.97     12500
         pos       0.97      0.97      0.97     12500

    accuracy                           0.97     25000
   macro avg       0.97      0.97      0.97     25000
weighted avg       0.97      0.97      0.97     25000



array([[12065,   435],
       [  326, 12174]])

### **Conclusion**
we have seen how much easy to use ktrain library to tuned the bert model. Here, used samller bert model to predict the out put of IMDB movide reviews. Also, we got 97% accuracy.